In [15]:
file_path = '/content/gdrive/My Drive/Analitica/Proyecto 3/Nuevo_Archivo.csv'
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

data = df = pd.read_csv(file_path)

# 1. Eliminar filas con valores faltantes
data_cleaned = data.dropna()

# 2. Identificar columnas categóricas
categorical_cols = data_cleaned.select_dtypes(include=['object']).columns

# Aplicar Label Encoding para columnas categóricas binarias
label_encoded_cols = []
label_encoder = LabelEncoder()

for col in categorical_cols:
    if data_cleaned[col].nunique() <= 2:
        data_cleaned[col] = label_encoder.fit_transform(data_cleaned[col])
        label_encoded_cols.append(col)

# 3. Aplicar One-Hot Encoding para columnas categóricas con más de 2 categorías
data_final = pd.get_dummies(data_cleaned, columns=[col for col in categorical_cols if col not in label_encoded_cols])


<ipython-input-16-9a806ace6a51>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned[col] = label_encoder.fit_transform(data_cleaned[col])
<ipython-input-16-9a806ace6a51>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned[col] = label_encoder.fit_transform(data_cleaned[col])
<ipython-input-16-9a806ace6a51>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

In [17]:
import pandas as pd
import numpy as np
import re

def unify_family_members(data_final):
    """
    Combina columnas que comienzan con 'FAMI_PERSONASHOGAR_' en una sola columna 'FAMI_PERSONASHOGAR'.
    Si una columna tiene valor True, se asignará el número correspondiente al nombre de la columna.

    Args:
        data_final: DataFrame con columnas que comienzan con 'FAMI_PERSONASHOGAR_'.

    Returns:
        DataFrame actualizado con la columna unificada y las columnas originales eliminadas.
    """
    # Identificar todas las columnas que comienzan con 'FAMI_PERSONASHOGAR_'
    family_cols = [col for col in data_final.columns if col.startswith('FAMI_PERSONASHOGAR_')]

    # Crear un diccionario para convertir números en texto a números enteros
    text_to_num = {
        'una': 1, 'uno': 1, 'dos': 2, 'tres': 3, 'cuatro': 4, 'cinco': 5,
        'seis': 6, 'siete': 7, 'ocho': 8, 'nueve': 9, 'diez': 10, 'once': 11,
        'doce': 12, 'trece': 13, 'catorce': 14, 'quince': 15
    }

    # Crear una nueva columna 'FAMI_PERSONASHOGAR' inicializada con NaN
    data_final['FAMI_PERSONASHOGAR'] = np.nan

    # Iterar sobre las columnas que coincidan
    for col in family_cols:
        # Extraer el número desde el nombre de la columna
        match = re.search(r'_(\w+)$', col)
        if match:
            num_text = match.group(1).lower()  # Extraer y convertir a minúsculas
            if num_text in text_to_num:
                number = text_to_num[num_text]
                # Donde el valor sea True, asignar el número a 'FAMI_PERSONASHOGAR'
                data_final.loc[data_final[col] == True, 'FAMI_PERSONASHOGAR'] = number

    # Eliminar las columnas originales
    data_final = data_final.drop(columns=family_cols)

    # Opcional: Rellenar valores NaN con 0
    data_final['FAMI_PERSONASHOGAR'] = data_final['FAMI_PERSONASHOGAR'].fillna(0)

    return data_final

# Aplicar la función a tu DataFrame final
data_final = unify_family_members(data_final)

# Verificar los resultados
print(data_final.head())


   COLE_DEPTO_UBICACION  COLE_BILINGUE  COLE_NATURALEZA  FAMI_TIENEAUTOMOVIL  \
4                     0              0                1                    0   
5                     0              0                1                    0   
6                     0              0                1                    1   
7                     0              0                1                    1   
8                     0              0                1                    1   

   FAMI_TIENECOMPUTADOR  FAMI_TIENEINTERNET  FAMI_TIENELAVADORA  PUNT_GLOBAL  \
4                     0                   0                   1        274.0   
5                     1                   1                   1        244.0   
6                     1                   1                   1        222.0   
7                     1                   1                   1        348.0   
8                     1                   1                   1        348.0   

   PUNT_INGLES  PUNT_MATEMATICAS  ... 

In [18]:
def unify_estrato_vivienda(data_final):
    """
    Combina columnas que comienzan con 'FAMI_ESTRATOVIVIENDA_Estrato' en una sola columna 'FAMI_ESTRATOVIVIENDA'.
    Si una columna tiene valor True, se asignará el número correspondiente al nombre de la columna.

    Args:
        data_final: DataFrame con columnas que comienzan con 'FAMI_ESTRATOVIVIENDA_Estrato'.

    Returns:
        DataFrame actualizado con la columna unificada y las columnas originales eliminadas.
    """
    # Identificar todas las columnas que comienzan con 'FAMI_ESTRATOVIVIENDA_Estrato'
    estrato_cols = [col for col in data_final.columns if col.startswith('FAMI_ESTRATOVIVIENDA_Estrato')]

    # Crear una nueva columna 'FAMI_ESTRATOVIVIENDA' inicializada con NaN
    data_final['FAMI_ESTRATOVIVIENDA'] = np.nan

    # Iterar sobre las columnas que coincidan
    for col in estrato_cols:
        # Extraer el número del estrato desde el nombre de la columna
        match = re.search(r'Estrato_(\d+)$', col)
        if match:
            estrato = int(match.group(1))  # Convertir a número entero
            # Donde el valor sea True, asignar el número a 'FAMI_ESTRATOVIVIENDA'
            data_final.loc[data_final[col] == True, 'FAMI_ESTRATOVIVIENDA'] = estrato

    # Eliminar las columnas originales
    data_final = data_final.drop(columns=estrato_cols)

    # Opcional: Rellenar valores NaN con 0
    data_final['FAMI_ESTRATOVIVIENDA'] = data_final['FAMI_ESTRATOVIVIENDA'].fillna(0)

    return data_final

# Aplicar la función a tu DataFrame
data_final = unify_estrato_vivienda(data_final)

# Verificar los resultados
print(data_final.head())


   COLE_DEPTO_UBICACION  COLE_BILINGUE  COLE_NATURALEZA  FAMI_TIENEAUTOMOVIL  \
4                     0              0                1                    0   
5                     0              0                1                    0   
6                     0              0                1                    1   
7                     0              0                1                    1   
8                     0              0                1                    1   

   FAMI_TIENECOMPUTADOR  FAMI_TIENEINTERNET  FAMI_TIENELAVADORA  PUNT_GLOBAL  \
4                     0                   0                   1        274.0   
5                     1                   1                   1        244.0   
6                     1                   1                   1        222.0   
7                     1                   1                   1        348.0   
8                     1                   1                   1        348.0   

   PUNT_INGLES  PUNT_MATEMATICAS  ... 

In [22]:
def convert_boolean_to_numeric(df):
    """
    Convierte todas las columnas con valores booleanos (True/False) a valores numéricos (1/0).

    Args:
        df: DataFrame con columnas booleanas.

    Returns:
        DataFrame con valores numéricos en lugar de booleanos.
    """
    # Encontrar columnas booleanas
    bool_cols = df.select_dtypes(include='bool').columns

    # Convertir las columnas booleanas a valores numéricos
    df[bool_cols] = df[bool_cols].astype(int)

    return df

# Aplicar la función al DataFrame
data_final = convert_boolean_to_numeric(data_final)




In [23]:
data_final.head()

,COLE_DEPTO_UBICACION,COLE_BILINGUE,COLE_NATURALEZA,FAMI_TIENEAUTOMOVIL,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,PUNT_GLOBAL,PUNT_INGLES,PUNT_MATEMATICAS,...,FAMI_EDUCACIONPADRE_Postgrado,FAMI_EDUCACIONPADRE_Primaria completa,FAMI_EDUCACIONPADRE_Primaria incompleta,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) completa,FAMI_EDUCACIONPADRE_Secundaria (Bachillerato) incompleta,FAMI_EDUCACIONPADRE_Técnica o tecnológica completa,FAMI_EDUCACIONPADRE_Técnica o tecnológica incompleta,FAMI_ESTRATOVIVIENDA_Sin Estrato,FAMI_PERSONASHOGAR,FAMI_ESTRATOVIVIENDA
4,0,0,1,0,0,0,1,274.0,50.0,60.0,...,0,0,1,0,0,0,0,0,0.0,0.0
5,0,0,1,0,1,1,1,244.0,40.0,48.0,...,0,0,0,1,0,0,0,0,0.0,0.0
6,0,0,1,1,1,1,1,222.0,44.0,48.0,...,0,0,0,0,1,0,0,0,0.0,0.0
7,0,0,1,1,1,1,1,348.0,60.0,75.0,...,0,1,0,0,0,0,0,0,0.0,0.0
8,0,0,1,1,1,1,1,348.0,60.0,75.0,...,0,1,0,0,0,0,0,0,0.0,0.0


In [24]:
# prompt: descargue ese ultimo df en mi compu y llamo dfFinal

#from google.colab import files
#data_final.to_csv('dfFinal.csv', encoding = 'utf-8-sig')
#files.download('dfFinal.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>